In [ ]:
# --------------------------------------------------------------
#  DRAWING BOARD NOTEBOOK – run while training is still going on
# --------------------------------------------------------------
import tkinter as tk
from tkinter import ttk, messagebox
from PIL import Image, ImageOps, ImageDraw
import numpy as np
import tensorflow as tf
from pathlib import Path

In [ ]:
# ---------- CHANGE ONLY IF YOUR FOLDER STRUCTURE IS DIFFERENT ----------
NOTEBOOK_ROOT = Path(r"C:\Users\TIK03\Documents\GitHub\DIT5411-HoYiTik\Assgnment")
MODELS_DIR    = NOTEBOOK_ROOT / "saved_models"
TRAIN_DIR     = NOTEBOOK_ROOT / "processed_data" / "train"   # needed for class names

IMG_SIZE   = (64, 64)                 # must match training
BEST_MODEL = MODELS_DIR / "mlp_baseline.h5"
# --------------------------------------------------------------------

In [ ]:
# ---- load the model (will wait for the .h5 file to appear) ----
import time, os

def wait_for_model(path, timeout=600):
    """Wait up to `timeout` seconds for the model file to exist."""
    print(f"Waiting for model: {path.name} ...")
    start = time.time()
    while not path.exists():
        if time.time() - start > timeout:
            raise TimeoutError(f"Model {path} not found after {timeout}s")
        time.sleep(2)
    print(f"Model found after {time.time()-start:.1f}s")

wait_for_model(BEST_MODEL)
model = tf.keras.models.load_model(str(BEST_MODEL))
print("Model loaded successfully")

# ---- class names (folder names in train_dir) ----
class_names = sorted([p.name for p in TRAIN_DIR.iterdir() if p.is_dir()])
NUM_CLASSES = len(class_names)
print(f"Detected {NUM_CLASSES} classes")

In [ ]:
def preprocess_drawing(pil_img: Image.Image) -> np.ndarray:
    """64×64 PIL → normalized (1,64,64,1) tensor."""
    img = pil_img.convert('L')
    img = ImageOps.invert(img)                 # dark strokes on light bg
    img = ImageOps.fit(img, IMG_SIZE, method=Image.BILINEAR)
    arr = np.array(img, dtype=np.float32) / 255.0
    arr = np.expand_dims(arr, axis=[0, -1])    # (1,64,64,1)
    return arr

In [ ]:
# --------------------------------------------------------------
#  Cell 5 – Updated DrawingBoard (fixed empty-canvas check)
# --------------------------------------------------------------
class DrawingBoard(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Live Character Predictor")
        self.resizable(False, False)

        self.scale = 8                                 # 512x512 window
        self.canvas_size = (IMG_SIZE[0]*self.scale, IMG_SIZE[1]*self.scale)
        self.canvas = tk.Canvas(self, width=self.canvas_size[0],
                                height=self.canvas_size[1], bg='black')
        self.canvas.pack(padx=10, pady=10)

        # hidden 64x64 image (white background)
        self.pil_img = Image.new('L', IMG_SIZE, 255)
        self.draw = ImageDraw.Draw(self.pil_img)

        self.last_x = self.last_y = None
        self.brush = 5                                 # logical pixels

        self.canvas.bind("<B1-Motion>", self.paint)
        self.canvas.bind("<ButtonRelease-1>", self.reset)

        # buttons
        btns = ttk.Frame(self)
        btns.pack(pady=5)
        ttk.Button(btns, text="Clear", command=self.clear).grid(row=0, column=0, padx=4)
        ttk.Button(btns, text="Predict", command=self.predict).grid(row=0, column=1, padx=4)
        ttk.Button(btns, text="Quit", command=self.destroy).grid(row=0, column=2, padx=4)

        # result label
        self.result_var = tk.StringVar(value="Draw → Predict")
        ttk.Label(self, textvariable=self.result_var, font=("Arial", 12)).pack(pady=8)

    # ----------------------------------------------------------
    def paint(self, e):
        x, y = e.x // self.scale, e.y // self.scale
        if self.last_x is not None:
            # visible canvas (scaled)
            self.canvas.create_line(
                self.last_x*self.scale, self.last_y*self.scale,
                e.x, e.y,
                fill='white', width=self.brush*self.scale, capstyle=tk.ROUND)
            # hidden 64x64 image
            self.draw.line([self.last_x, self.last_y, x, y],
                           fill=0, width=self.brush, joint='curve')
        self.last_x, self.last_y = x, y

    def reset(self, _=None):
        self.last_x = self.last_y = None

    def clear(self):
        self.canvas.delete("all")
        self.pil_img = Image.new('L', IMG_SIZE, 255)
        self.draw = ImageDraw.Draw(self.pil_img)
        self.result_var.set("Canvas cleared")

    # ----------------------------------------------------------
    def predict(self):
        # ---- NEW EMPTY-CANVAS CHECK ----
        img_arr = np.array(self.pil_img)
        if img_arr.min() == 255:                # no pixel darker than white → empty
            messagebox.showinfo("Empty", "Draw something first!")
            return

        X = preprocess_drawing(self.pil_img)
        probs = model.predict(X, verbose=0)[0]
        top5_i = np.argsort(probs)[-5:][::-1]
        top5_n = [class_names[i] for i in top5_i]
        top5_c = probs[top5_i]

        pred = top5_n[0]
        conf = " | ".join(f"{n}: {c:.2%}" for n,c in zip(top5_n, top5_c))
        self.result_var.set(f"Prediction: {pred}\nTop-5: {conf}")

# ----------------------------------------------------------
print("\n=== Drawing board ready (fixed) ===\nDraw with left mouse → Predict")
DrawingBoard().mainloop()